In [ ]:
# Enable autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from tqdm.auto import tqdm
import pickle
from pathlib import Path
import numpy as np
import pgeocode
import datetime
import re
from datetime import datetime

pd.set_option('max_columns',None)

# Import Raw Data

Import Raw Data from Excel files

In [ ]:
raw_path = Path('./data/export_raw_rech_2016_2020.pkl.bz2')

if raw_path.exists():
    df_raw = pd.read_pickle(raw_path)
else:
    files = [
        './data/raw/export_GJ2016.xlsx', 
        './data/raw/export_GJ2017.xlsx', 
        './data/raw/export_GJ2018.xlsx',
        './data/raw/export_GJ2019.xlsx',
        './data/raw/export_GJ2020.xlsx'
    ]

    df_list = []

    for file in tqdm(files):
        print('Importing file "{}"'.format(file))
        df_tmp = pd.read_excel(file)
        df_list.append(df_tmp)
    
    df_raw = pd.concat(df_list)
    df_raw.to_pickle('./data/export_raw_rech_2016_2020.pkl.bz2')

display(df_raw.head())

In [ ]:
df = df_raw.copy()
display(df_raw.isna().sum())

# Clean Raw Data

## Remove Segment 1, 13 and entries older than 2014

In [ ]:
# VERANST_SEGMENT is between 2 and 12
df=df[(~df['VERANST_SEGMENT'].isnull()) & (~df['VERANST_SEGMENT'].isin([1,13]))]

#VG_DATUM_VON - convert to datetime and drop entries with year < 2014
  
df['VG_DATUM_VON'] = pd.to_datetime(df['VG_DATUM_VON'], format='%Y%m%d')
df=df[df['VG_DATUM_VON']>'2014-01-01']

display(df.shape[0]/df_raw.shape[0])

In [ ]:
df = df.reset_index(drop=True)
df[df.index.duplicated()]

## Text cleansing function

In [ ]:
def clean_tarif(df_row):
    result = str(df_row)
    if result == "U-V III.1" :
        result = "U-V III. 1"
    elif result == "U-V III.3" :
        result = "U-V III. 3"
    elif result == "U-V II 1(+II 2A) KARAOKE" :
        result = "U-V II. 1 (+II 2A) KARAOKE"
    elif result == "U-V II. 1(+II 2)" :
        result = "U-V II. 1 (+II 2)"
    elif result == "U-ST I (MUSIKER)" :
        result = "U-ST I. (MUSIKER)"
    
    return result

print("Number of unique TARIF_BEZ before transformation {}".format(len(df.TARIF_BEZ.unique()))) 
df['TARIF_BEZ'] = df['TARIF_BEZ'].apply(clean_tarif)
print("Number of unique TARIF_BEZ after transformation {}".format(len(df.TARIF_BEZ.unique()))) 


In [ ]:
def clean_text(df_row):
    """
    Cleans text data
        * removes leading and trailing special characters
        * replaces empty string, (KEINE ANGABE) and single " with np.Nan

    Parameters
    ----------
    df_row : Series element

    Returns
    -------
    string
    """
    result = str(df_row)
    result = result.strip(" .*:/-,+&?;")
    if result in ['nan','','(KEINE ANGABE)', '"']:
        return np.nan

    return result


text_columns = [
    "VG_RAUM", "MUSIKLEITER_NAME", "KAPELLE_NAME", "VERANST_NAME", "VERANST_STRASSE", "VERANST_ORT","VERANST_PLZ"
]

for col in tqdm(text_columns):
    df[col] = df[col].apply(clean_text)

display((df[text_columns] != df[text_columns]).mean(axis=0))

#with pd.option_context("display.max_rows", 100):
#    display(df[col].value_counts()[:100])

## Replace Postcode with City Name

In [ ]:
plz_matcher = re.compile('^([0-9]{5})$')
nomi = pgeocode.Nominatim('de')


# clean VG_ORT
def clean_place(df_row):
    """
    Replace

    Parameters
    ----------
    df_row : Series element

    Returns
    -------
    string
    """
    result = str(df_row)
    if plz_matcher.match(result):
        geo_result = nomi.query_postal_code(result)
        result = geo_result.place_name.split(',')[0]
        result = result.replace('ß', 'SS')
        result = result.upper()
        result = result.replace('Ä', 'AE')
        result = result.replace('Ö', 'OE')
        result = result.replace('Ü', 'UE')
        print(result)

    return result


df['VG_ORT'] = df['VG_ORT'].apply(clean_text).fillna('').apply(clean_place).apply(clean_text)

display((df[['VG_ORT']] != df[['VG_ORT']]).mean(axis=0))

# Prepare for Feature Generation

## Create LOCATION column

In [ ]:
df['LOCATION']=df['VG_ORT'].fillna('')+' ' + df['VG_RAUM'].fillna('')
df['LOCATION']=df['LOCATION'].apply(clean_text)

display(df['LOCATION'].head(10))
display(df[['VG_ORT','VG_RAUM','LOCATION']].isna().sum())


## Create BAND column

In [ ]:
#Create BAND Column
df['BAND']=df['KAPELLE_NAME'].fillna(df['MUSIKLEITER_NAME'])

display(df['BAND'].dropna().head(10))
display(df[['KAPELLE_NAME','BAND']].isna().sum())

## Create PROMOTER column

In [ ]:
#Create PROMOTER Column
df['PROMOTER']=df['VERANST_NAME'].fillna('')+' ' + df['VERANST_PLZ'].fillna('')
df['PROMOTER']=df['PROMOTER'].apply(clean_text)

display(df['PROMOTER'].head(10))
display(df[['PROMOTER','VERANST_NAME','VERANST_PLZ']].isna().sum())


# Clean VG_INKASSO

In [ ]:
df.loc[df.VG_INKASSO == 0, 'VG_INKASSO'] = np.nan

#display(df.loc[df.VG_INKASSO.isnull()])
display('NaN values in df_raw : {}, NaN values in df: {}'.format(df_raw['VG_INKASSO'].isna().sum(), df['VG_INKASSO'].isna().sum()))

# Drop Duplicates

In [ ]:
df = df.dropna(subset=['VG_INKASSO'])
df = df.drop_duplicates(subset=['ID']).set_index('ID', verify_integrity=True).sort_index()

#convert certain columns to 'category'

df['IMPORT'] = df['IMPORT'].astype('category')
df['REKLA_JN'] = df['REKLA_JN'].astype('category')
df['TARIF_BEZ'] = df['TARIF_BEZ'].astype('category')
df['VG_ORT'] = df['VG_ORT'].astype('category')
df['VG_RAUM'] = df['VG_RAUM'].astype('category')

In [ ]:
display(df.shape[0]/df_raw.shape[0])
print(df.isna().sum())

# Store Prepared Data

Store prepared data in a pickle file for further usage

In [ ]:
df.to_pickle('./data/export_rech_2016_2020.pkl.bz2', protocol=4)